In [170]:
import dask
from dask.distributed import Client
import dask.dataframe as dd
from datetime import datetime
import numpy as np
import pandas as pd
from tqdm import trange,tqdm
import numba
from functools import lru_cache

import os
# client = Client(n_workers=12, threads_per_worker=4)

In [171]:
init_dtype= {
    'close': np.float32,
    'high': np.float32,
    'low': np.float32,
    'open': np.float32,
    'sma5': np.float32,
    'sma10': np.float32,
    'sma15': np.float32,
    'sma20': np.float32,
    'ema5': np.float32,
    'ema10': np.float32,
    'ema15': np.float32,
    'ema20': np.float32,
    'upperband': np.float32,
    'middleband': np.float32,
    'lowerband': np.float32,
    'HT_TRENDLINE': np.float32,
    'KAMA10': np.float32,
    'KAMA20': np.float32,
    'KAMA30': np.float32,
    'SAR': np.float32,
    'TRIMA5': np.float32,
    'TRIMA10': np.float32,
    'TRIMA20': np.float32,
    'ADX5': np.float32,
    'ADX10': np.float32,
    'ADX20': np.float32,
    'APO': np.float32,
    'CCI5': np.float32,
    'CCI10': np.float32,
    'CCI15': np.float32,
    'macd510': np.float32,
    'macd520': np.float32,
    'macd1020': np.float32,
    'macd1520': np.float32,
    'macd1226': np.float32,
    'MFI': np.float32,
    'MOM10': np.float32,
    'MOM15': np.float32,
    'MOM20': np.float32,
    'ROC5': np.float32,
    'ROC10': np.float32,
    'ROC20': np.float32,
    'PPO': np.float32,
    'RSI14': np.float32,
    'RSI8': np.float32,
    'slowk': np.float32,
    'slowd': np.float32,
    'fastk': np.float32,
    'fastd': np.float32,
    'fastksr': np.float32,
    'fastdsr': np.float32,
    'ULTOSC': np.float32,
    'WILLR': np.float32,
    'ATR': np.float32,
    'Trange': np.float32,
    'TYPPRICE': np.float32,
    'HT_DCPERIOD': np.float32,
    'BETA': np.float32,
    'volume': np.uint32,
    'date': 'string[pyarrow]'
}
dtype_cast = {
    'close': np.float32,
    'high': np.float32,
    'low': np.float32,
    'open': np.float32,
    'sma5': np.float32,
    'sma10': np.float32,
    'sma15': np.float32,
    'sma20': np.float32,
    'ema5': np.float32,
    'ema10': np.float32,
    'ema15': np.float32,
    'ema20': np.float32,
    'upperband': np.float32,
    'middleband': np.float32,
    'lowerband': np.float32,
    'HT_TRENDLINE': np.float32,
    'KAMA10': np.float32,
    'KAMA20': np.float32,
    'KAMA30': np.float32,
    'SAR': np.float32,
    'TRIMA5': np.float32,
    'TRIMA10': np.float32,
    'TRIMA20': np.float32,
    'ADX5': np.float32,
    'ADX10': np.float32,
    'ADX20': np.float32,
    'APO': np.float32,
    'CCI5': np.float32,
    'CCI10': np.float32,
    'CCI15': np.float32,
    'macd510': np.float32,
    'macd520': np.float32,
    'macd1020': np.float32,
    'macd1520': np.float32,
    'macd1226': np.float32,
    'MFI': np.float32,
    'MOM10': np.float32,
    'MOM15': np.float32,
    'MOM20': np.float32,
    'ROC5': np.float32,
    'ROC10': np.float32,
    'ROC20': np.float32,
    'PPO': np.float32,
    'RSI14': np.float32,
    'RSI8': np.float32,
    'slowk': np.float32,
    'slowd': np.float32,
    'fastk': np.float32,
    'fastd': np.float32,
    'fastksr': np.float32,
    'fastdsr': np.float32,
    'ULTOSC': np.float32,
    'WILLR': np.float32,
    'ATR': np.float32,
    'Trange': np.float32,
    'TYPPRICE': np.float32,
    'HT_DCPERIOD': np.float32,
    'BETA': np.float32,
    'volume': np.uint32
}

PATH = 'd2/src/'

files= [PATH+fp for fp in os.listdir('d2/src/') if '.csv' in fp]
names = [n[7:-32] for n in files]

@numba.jit()
def slicer_vectorized(a,start=0,end=19):
    b = a.view((str,1)).reshape(len(a),-1)[:,start:end]
    return np.fromstring(b.tostring(),dtype=(str,end-start))

In [172]:
%%time
for file,name in tqdm(zip(files,names)):
    ddf = dd.read_csv(file,dtype=init_dtype)
    y_ = ddf.date.compute()
    dates = slicer_vectorized(y.to_numpy(dtype=str))
    dates = dates.astype(np.datetime64)
    ddf.dates = pd.Series(dates,index=y.index)
    ddf = ddf.set_index('date')
    ddf.to_parquet('d2/processed/',name_function=lambda i: f"{name}-{i}.parquet",compression='zstd')


101it [15:34,  9.25s/it]

CPU times: user 3min 11s, sys: 22.6 s, total: 3min 33s
Wall time: 15min 34s


In [153]:
# files= [PATH+fp for fp in os.listdir('d2/src/') if '.csv' in fp]


tar -c -I 'xz -9 -T0' -f archive.tar.xz processed

'EICHERMOT'

In [156]:
%%time


CPU times: user 514 ms, sys: 126 ms, total: 639 ms
Wall time: 4.34 s


In [163]:
!pip install brotli

Defaulting to user installation because normal site-packages is not writeable
  Using cached Brotli-1.0.9-cp39-cp39-manylinux1_x86_64.whl (357 kB)


In [127]:
dkdf.memory_usage(deep=True).compute().sum()/10**6

177.82211

In [134]:
dkdf3.memory_usage(deep=True).compute().sum()/10**6

177.82242

In [2]:
def metaDB(object):
    pass

In [ ]:
for file in files:
    dk

In [96]:
# dkdf = dd.read_csv(files[0])

In [32]:
# y = dkdf.date.compute()
dates = dates.astype(np.datetime64)


In [168]:
17*9


153

In [ ]:
dd.read_csv()

In [9]:
from numba import jit
import warnings
warnings.filterwarnings('ignore')

In [76]:
dkdf.astype(dtype_cast)

,date,close,high,low,open,volume,sma5,sma10,sma15,sma20,ema5,ema10,ema15,ema20,upperband,middleband,lowerband,HT_TRENDLINE,KAMA10,KAMA20,KAMA30,SAR,TRIMA5,TRIMA10,TRIMA20,ADX5,ADX10,ADX20,APO,CCI5,CCI10,CCI15,macd510,macd520,macd1020,macd1520,macd1226,MFI,MOM10,MOM15,MOM20,ROC5,ROC10,ROC20,PPO,RSI14,RSI8,slowk,slowd,fastk,fastd,fastksr,fastdsr,ULTOSC,WILLR,ATR,Trange,TYPPRICE,HT_DCPERIOD,BETA
npartitions=10,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,float32,float32,float32,float32,uint32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [83]:
dkdf.memory_usage().compute().sum()

322094720